Run the config file to authenticate script and query Key Vault

Second script. Run after 'Query_Activites'

In [32]:
%run config

Code from file 'file:///c%3A/Users/nicholas.radich/Documents/Strava_Lakehouse/config.py':
 client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 
client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 
new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")
activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 
historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 
segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 



import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'client_id':  client_id,
    'client_secret': client_secret,
    'refresh_token': new_refresh_token,
    

Dependencies 

In [33]:
from pyspark.sql.functions import * 
from pyspark.sql import functions as F
from pyspark.sql import Row
import pandas as pd

### API Query to get more specific details for each activity, need to pass each activity off individually 

Get full activity dataset from what is written in storage, should be all activites

In [34]:

full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

Grab all of the disinct activity IDs 

In [35]:
full_activity_ids = full_activity_dataset.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

In [36]:
def query_segments(activity_ids : list):
    """Gets all segment_ids for each activity_id submitted
    Returns distinct values"""
    df = pd.DataFrame()
    activity_id_list =[]
    segment_id_list =[]
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)
                  
            columns = ['segment_id', 'activity_id']
            extracted_data = [segment_id_list, activity_id_list]
            segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))

            df = pd.concat([df, segment_df])

            count += 1

    #convert pandas df to spark
        
    segment_spark_df = spark.createDataFrame(df)

    #drop duplicate entries
    segment_spark_df = segment_spark_df.dropDuplicates()

    segment_spark_df = segment_spark_df.select(concat(segment_spark_df.segment_id,segment_spark_df.activity_id).alias("Activity_Segment_JointID"), 'segment_id','activity_id')

    segment_spark_df = segment_spark_df.withColumn("ingest_file_name", lit("segment_efforts_ids")) \
                                .withColumn("ingested_at", lit(current_timestamp()))

    return segment_spark_df

In [37]:
#will need to compare the activity ids that have already been queried for their segments
segments_in_storage = spark.read.format("delta").load(segment_effort_path)
activity_ids_with_queried_segments = segments_in_storage.select('activity_id').distinct().rdd.flatMap(lambda x: x).collect()


activity_ids_without_queried_segments = [x for x in full_activity_ids  if x not in activity_ids_with_queried_segments ]

#grab the first 99 spots so as not to overload the api call
eligible_activities = activity_ids_without_queried_segments[:99]



segment_id_df = query_segments(eligible_activities)

In [38]:
segment_id_df.show(20)


+------------------------+-------------------+-----------+-------------------+--------------------+
|Activity_Segment_JointID|         segment_id|activity_id|   ingest_file_name|         ingested_at|
+------------------------+-------------------+-----------+-------------------+--------------------+
|    30624788126478716...|3062478812647871650| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|
|    30624788126487051...|3062478812648705186| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|
|    30624788126478399...|3062478812647839906| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|
|    30624788126491076...|3062478812649107618| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|
|    30624788126513194...|3062478812651319458| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|
|    30624788126477528...|3062478812647752866| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|
|    30624788126488342...|3062478812648834210| 8589595490|segment_efforts_ids|2023-08-16 19:32:...|


In [43]:
value_counts = segment_id_df.groupBy("ingested_at").agg(F.count("ingested_at").alias("count")).orderBy("count", ascending=False)
value_counts.show()

+--------------------+-----+
|         ingested_at|count|
+--------------------+-----+
|2023-08-16 19:34:...| 1472|
+--------------------+-----+

In [49]:
segment_id_df.select("activity_id").distinct().count()

73

In [28]:
for i in eligible_activities:
    print(i)

9153279740
5972584497
7166250297
8589595490
8146999482
9122668027
5740260874
8789168949
8304389973
4838912326
9097857822
8969740420
3809768344
4079983225
6129061619
8800628187
8653447163
9235757648
8834338946
5718648252
7041321207
9217399958
7218381768
5987842682
6040654473
3865986737
7062090532
7208614563
9248492217
8595916872
8155505699
8987499152
3764074685
3871052555
8435154163
8268708556
4768097716
9018676993
8667539340
7224011708
4414021787
8928195891
7095425818
8360928782
9012852102
5997390802
6114231607
6059385275
8275855602
8377663140
6992488859
5524575370
3876332481
8633838220
4730507502
5755543494
4382047703
9197541512
7138047360
8159479241
6991844521
8420302190
7004837267
8178281092
4910436714
7229384684
7358811380
7743374567
5448078140
6395162679
5408255638
7261304130
7512286724
7411376407
7303733207
5391043545
6302328393
7491632820
6270833770
6284670536
7728807247
7815243449
5198781723
4993537055
5326544918
5281545880
7405131726
7701046541
7399680604
6239126619
7523741608

Take unique activity ids, and extract all of the segments associated with those activities

Need to compare activites already stored with segments as there are limits for strava API

Initially will go 99 request, but might need to reduce that to save requests for segments

Query all segments from all activities 

Not all activities register segments, will need to append into final DF that is writtent to storage with placerholder values so as not to keep querying them

In [ ]:
segment_id_df.printSchema()

In [ ]:
from pyspark.sql import Row
#activities that returned segments
#need to add in the lambda to get the row values
returned_activity_ids = segment_id_df.select("activity_id").distinct().rdd.flatMap(lambda x: x).collect()

#activies submitted, that did not return segments
activity_ids_without_segments = [x for x in test_id_list if x not in returned_activity_ids ]

#Append in activies without segments to the DF
#convert the list of ids_without segments into a DF
#need to adjust column names 
rows = [Row(Activity_Segment_JointID=i,  activity_id = i) for i in activity_ids_without_segments]
new_df = spark.createDataFrame(rows)
new_df = new_df.withColumn("ingest_file_name", lit("segment_efforts_ids")) \
                                .withColumn("ingested_at", lit(current_timestamp()))\
                                .withColumn("segment_id", lit(None).cast("long"))

#reorder columns to union into 
new_df_reordered = new_df.select(*segment_id_df.columns)

#union the two datasets together
all_segment_ids = segment_id_df.union(new_df_reordered)

In [ ]:
all_segment_ids.show()

In [ ]:
def write_dataframe_to_storage(dataset, storage_path, option, mode ):
    """Function to write activity ids to storage. Will overwrite current delta file in storage
    Option refers to schema overwriteSchema or mergeSchema, mode being either overwrite or append"""
    dataset.write.format("delta")\
    .option(option, "true")\
    .mode(mode)\
    .save(storage_path)

In [ ]:
segments_in_storage = spark.read.format("delta").load(segment_effort_path)

In [ ]:
write_dataframe_to_storage(all_segment_ids,segment_effort_path, "overwriteSchema", "overwrite" )

In [ ]:
segments_in_storage.show(10)

In [ ]:
try:
    #Query path, see if there are any activities with their associated segments written to storage
    segments_in_storage = spark.read.format("delta").load(segment_effort_path)
except:
    #if that errors, meaning first time running the script
    #Write the first 99 activites to storage, will need to specificy sort order 
    top_99_activity_ids = 
    write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )


Need to limit to 100 request as the api throws an errors after

In [ ]:
write_dataframe_to_storage(segment_id_df,segment_effort_path, "overwriteSchema","overwrite" )

In [ ]:
segments_in_storage = spark.read.format("delta").load(segment_effort_path)

In [ ]:
#need to get distinct activity_ids and run them through the segment
#we make 1 api query in the first script, so gonna be allowed 99 with this run 

In [ ]:
#get all of the activity_ids, limit to 15 as thats how many we can run in a single query
#also need to query the activities we have written to segment storage, so as not to repeat 
#Strava API usage is limited on a per-application basis using both a 15-minute and daily request limit. The default rate limit allows 100 requests every 15 minutes, 
# with up to 1,000 requests per day.

#compare current activites vs what is written, 
#going to need to write some try and excepts for expecting these values in return 